In [2]:
import numpy 
from matplotlib import pyplot
from amuse.units import units, constants
from amuse.lab import nbody_system
from amuse.couple import bridge
from amuse.community.huayno import Huayno
from amuse.ic.salpeter import new_salpeter_mass_distribution
from amuse.ic.plummer import new_plummer_model
from amuse.ic.fractalcluster import new_fractal_cluster_model
from amuse.lab import Particle, Particles, units, nbody_system
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import math, numpy
from matplotlib import pyplot
from amuse.lab import *
from optparse import OptionParser
from amuse.ext.LagrangianRadii import LagrangianRadii
from amuse.community.ph4.interface import ph4

/home/zuzia/miniforge3/lib/python3.12/site-packages/amuse/support/options.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
class MilkyWay_galaxy(object):
    def __init__(
        self, 
        Mb=1.40592e10 | units.MSun,
        Md=8.5608e10 | units.MSun,
        Mh=1.07068e11 | units.MSun,
    ):
        self.Mb = Mb
        self.Md = Md
        self.Mh = Mh

    def get_potential_at_point(self,eps,x,y,z):
        r = (x**2+y**2+z**2)**0.5
        R = (x**2+y**2)**0.5
        # bulge
        b1 = 0.3873 | units.kpc
        pot_bulge = -constants.G*self.Mb/(r**2+b1**2)**0.5 
        # disk
        a2 = 5.31 | units.kpc
        b2 = 0.25 | units.kpc
        pot_disk = -constants.G*self.Md/(R**2 + (a2+ (z**2+ b2**2)**0.5 )**2 )**0.5
        # halo
        a3 = 12.0 | units.kpc
        cut_off = 100 | units.kpc
        d1= r/a3
        c = 1+ (cut_off/a3)**1.02
        pot_halo = (
            -constants.G * (self.Mh/a3) * d1**1.02/(1+ d1**1.02)
            -(constants.G*self.Mh/(1.02*a3))
            * (
                -1.02 / c + numpy.log(c) + 1.02/(1+d1**1.02) 
                - numpy.log(1.0 + d1**1.02)
            )
        )
        return 2*(pot_bulge+pot_disk+ pot_halo)  # multiply by 2 because it is a rigid potential
    
    def get_gravity_at_point(self, eps, x,y,z): 
        r = (x**2+y**2+z**2)**0.5
        R = (x**2+y**2)**0.5
        # bulge
        b1 = 0.3873 | units.kpc
        force_bulge = -constants.G*self.Mb/(r**2+b1**2)**1.5 
        # disk
        a2 = 5.31 | units.kpc
        b2 = 0.25 | units.kpc
        d = a2+ (z**2+ b2**2)**0.5
        force_disk = -constants.G*self.Md/(R**2+ d**2 )**1.5
        # halo
        a3 = 12.0 | units.kpc
        d1 = r/a3
        force_halo = -constants.G*self.Mh*d1**0.02/(a3**2*(1+d1**1.02))
       
        ax = force_bulge*x + force_disk*x  + force_halo*x/r
        ay = force_bulge*y + force_disk*y  + force_halo*y/r
        az = force_bulge*z + force_disk*d*z/(z**2 + b2**2)**0.5 + force_halo*z/r 

        return ax,ay,az

In [10]:
#setting the initial conditions - number of stars, velocity of the cluster, size of a cluster, position of a cluster
number_of_stars = 100
vel_cluster = 40

masses = new_salpeter_mass_distribution(number_of_stars, 1|units.MSun, 30|units.MSun)
#set to a 1000 particles cluster 
converter = nbody_system.nbody_to_si(masses.sum(), 1|units.parsec)

cluster = new_fractal_cluster_model(number_of_stars, converter)
cluster.mass = masses
cluster.move_to_center()
cluster.position += [8.5, 0, 0] | units.kpc
cluster.velocity += [0, vel_cluster, 0] |units.kms
# Assign a simple integer ID 0, 1, 2... to easily identify them later
cluster.id = range(len(cluster))

# Verify the massive ones
print(f"Most massive star ID: {cluster[cluster.mass.argmax()].id} (Mass: {cluster.mass.max().in_(units.MSun)})")


# print(cluster.x.value_in(units.parsec).min())
# print(cluster.x.value_in(units.parsec).max())

# print(cluster[2].mass)
cluster.radius = 500 | units.AU
print(cluster.radius.in_(units.AU))
print(cluster.virial_radius().in_(units.AU))

Most massive star ID: 83 (Mass: 19.6076696602 MSun)
[500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0] au
204295.461128 au


In [11]:
cluster_star_mass = []
for i in range(len(cluster)):
    cluster_star_mass.append(cluster[i].mass.value_in(units.MSun))

In [12]:
Mtot_init = cluster.mass.sum()
print("Total cluster mass:", Mtot_init)
Qvir = 0.5
cluster.scale_to_standard(converter, virial_ratio=Qvir)

gravity_code = ph4(converter, number_of_workers=1)
gravity_code.particles.add_particles(cluster)

channel_from_gd_to_framework = gravity_code.particles.new_channel_to(cluster)


stopping_condition = gravity_code.stopping_conditions.collision_detection
stopping_condition.enable()

MWG = MilkyWay_galaxy()

gravity = bridge.Bridge(use_threading=False)
gravity.add_system(gravity_code, (MWG,) )




t_end = 1 * 1e6 * 365.25 * 24 * 3600 | units.s  # 10 Myr w sekundach
nsteps = 10
dt = t_end / nsteps

time = 0.0 | units.s

gravity.timestep = dt / 2  # or simply 0.05 | units.Myr


Total cluster mass: 5.49647345612e+32 kg


Bove is chat version but I feel both mine and this have the same problem, My just belowe this cell

In [13]:

while time < t_end:
    target_time = time + dt
    print(f"Evolving to target: {target_time.in_(units.Myr)}")
    
    # 1. Ask Bridge to evolve
    gravity.evolve_model(target_time)

    # 2. Check for stopping conditions
    if stopping_condition.is_set():
        print(f"Collision detected at t={gravity.model_time.in_(units.Myr)}")
        
        # 3. Retrieve the colliding pairs
        # particles(0) gives the first particle of each pair
        # particles(1) gives the second particle of each pair
        p1s = stopping_condition.particles(0)
        p2s = stopping_condition.particles(1)
        
        particles_to_remove = Particles()
        
        # Check if we actually got any particles
        if len(p1s) > 0:
            for p1, p2 in zip(p1s, p2s):
                # Check if these particles are already in our "to remove" list
                # (in case of multi-collisions)
                if p1 not in particles_to_remove:
                    particles_to_remove.add_particle(p1)
                if p2 not in particles_to_remove:
                    particles_to_remove.add_particle(p2)
            
            print(f"Removing {len(particles_to_remove)} particles.")
            
            # Remove them from the N-body code
            gravity_code.particles.remove_particles(particles_to_remove)
            
            # Sync with framework channel if needed
            # channel_from_gd_to_framework.copy()
        else:
            # If is_set() is True but no particles are returned, it's a "ghost" trigger.
            # We just proceed.
            print("Stopping condition set, but no particles returned.")

        # 4. CRITICAL: RESET THE STOPPING CONDITION
        # Disabling and re-enabling clears the internal flag in most codes.
        stopping_condition.disable()
        stopping_condition.enable()
        
        # 5. SYNC THE TIME
        # Sometimes the bridge internal time needs to be updated to the code time
        # to prevent it from thinking it's still 'before' the collision.
        time = gravity.model_time

    else:
        # No collision, normal update
        time = gravity.model_time

    
    # Sync data back to python objects for analysis/plotting
    channel_from_gd_to_framework.copy()


Evolving to target: 0.10000213584301632 Myr
Collision detected at t=0.10000213584301632 Myr
Removing 2 particles.
Evolving to target: 0.20000427168603263 Myr
Collision detected at t=0.20000427168603263 Myr
Removing 2 particles.
Evolving to target: 0.30000640752904895 Myr
Collision detected at t=0.30000640752904895 Myr
Removing 2 particles.
Evolving to target: 0.40000854337206526 Myr
Collision detected at t=0.40000854337206526 Myr
Removing 2 particles.
Evolving to target: 0.5000106792150816 Myr
Collision detected at t=0.5000106792150816 Myr
Removing 2 particles.
Evolving to target: 0.6000128150580979 Myr
Collision detected at t=0.6000128150580979 Myr
Removing 2 particles.
Evolving to target: 0.7000149509011142 Myr
Collision detected at t=0.7000149509011142 Myr
Removing 2 particles.
Evolving to target: 0.8000170867441305 Myr
Collision detected at t=0.8000170867441305 Myr
Removing 2 particles.
Evolving to target: 0.9000192225871468 Myr
Collision detected at t=0.9000192225871468 Myr
Removi

This code is "working" but i think the particles do not feel the MW gravity for most of a time 

In [8]:
while time < t_end:
    com = gravity.particles.center_of_mass()
    print("Evolving to time:", time.in_(units.Myr))
    gravity.evolve_model(time + dt)

    t = gravity_code.model_time    
    while t < time + dt:
        t = gravity_code.model_time
        print(gravity_code.model_time.in_(units.Myr))
        if stopping_condition.is_set():
            print(f"  Collision #{i+1} at t={gravity_code.model_time.in_(units.Myr)}")
            part_set_1 = stopping_condition.particles(0)
            
            part_set_2 = stopping_condition.particles(1)
            
            for p1, p2 in zip(part_set_1, part_set_2):
                p1.radius = 0.1 | units.AU
                p2.radius = 0.1 | units.AU

        gravity_code.evolve_model(time + dt)

    time = gravity.model_time
            
            
            


Evolving to time: 0.0 Myr
0.0815145436673 Myr
  Collision #100 at t=0.0815145436673 Myr
0.0819876564165 Myr
  Collision #100 at t=0.0819876564165 Myr
0.0851602948522 Myr
  Collision #100 at t=0.0851602948522 Myr
0.0958053317087 Myr
  Collision #100 at t=0.0958053317087 Myr
0.096848962773 Myr
  Collision #100 at t=0.096848962773 Myr
0.100021601209 Myr
Evolving to time: 0.10000213584301632 Myr
0.105427610122 Myr
  Collision #100 at t=0.105427610122 Myr
0.109873478456 Myr
  Collision #100 at t=0.109873478456 Myr
0.117916395192 Myr
  Collision #100 at t=0.117916395192 Myr
0.118946111176 Myr
  Collision #100 at t=0.118946111176 Myr
0.125235727724 Myr
  Collision #100 at t=0.125235727724 Myr
0.136006000308 Myr
  Collision #100 at t=0.136006000308 Myr
0.136924395645 Myr
  Collision #100 at t=0.136924395645 Myr
0.139707411816 Myr
  Collision #100 at t=0.139707411816 Myr
0.148056460331 Myr
  Collision #100 at t=0.148056460331 Myr
0.157963997902 Myr
  Collision #100 at t=0.157963997902 Myr
0.179

In [ ]:
while time < t_end:
    com = gravity.particles.center_of_mass()
    # print(f"Cluster CoM: {com.in_(units.kpc)}")
    print("Evolving to time:", time.in_(units.Myr))
    
    gravity.evolve_model(time - dt)
    time = gravity.model_time
    print(gravity_code.model_time.in_(units.Myr))
    #print("--------------------------")
    # if time == 0.5 * 1e6 * 365.25 * 24 * 3600 | units.s:
    #     plt.scatter(cluster.x.value_in(units.parsec), cluster.y.value_in(units.parsec), s= 0.1)
    #     plt.xlabel("X (AU)")
    #     plt.ylabel("Y (AU)")
    #     plt.show()
    
    if stopping_condition.is_set():
        # print(" Collision detected!")
        #print(stopping_condition.particles)
        # 2. Iterate using zip to get pairs
        part_set_1 = stopping_condition.particles(0)
        print(len(part_set_1))
        part_set_2 = stopping_condition.particles(1)
        
        for p1, p2 in zip(part_set_1, part_set_2):
            # OPTION 1: Use ONLY .key (Removed .id)
            print(f"  Collision #{i+1} at t={gravity_code.model_time.in_(units.Myr)}")
            # print(f"  Collision Event:")
            
            # # p1.key is the unique ID generated by AMUSE. It persists across codes.
            #print(f"    Partner 1: AMUSE Key {p1.key}") 
            # print(f"               x={p1.x.in_(units.AU)}, vx={p1.vx.in_(units.kms)}")
            # print(f"               y={p1.y.in_(units.AU)}, vy={p1.vy.in_(units.kms)}")
            # print(f"               z={p1.z.in_(units.AU)}, vz={p1.vz.in_(units.kms)}")
            # print(f"               Mass={p1.mass.in_(units.MSun)}")
            
            #print(f"    Partner 2: AMUSE Key {p2.key}")
            # print(f"               x={p2.x.in_(units.AU)}, vx={p2.vx.in_(units.kms)}")
            # print(f"               y={p2.y.in_(units.AU)}, vy={p2.vy.in_(units.kms)}")
            # print(f"               z={p2.z.in_(units.AU)}, vz={p2.vz.in_(units.kms)}")
            # print(f"               Mass={p2.mass.in_(units.MSun)}")

            # Calculate relative velocity or distance
            dist = (p1.position - p2.position).length()
            # print(f"    Separation: {dist.in_(units.AU)}")
            if p1.mass > p2.mass:
                mass_ratio  = p1.mass / p2.mass
            else:
                mass_ratio  = p2.mass / p1.mass 
            # print(f"    Mass Ratio (larger/smaller): {mass_ratio}")

            p1.radius = 0.1 | units.AU
            p2.radius = 0.1 | units.AU
            gravity_code.evolve_model(time + dt)
       
        

print(cluster.radius.in_(units.AU))  



    # if stopping_condition.is_set():
    #     print(">> Collision detected!")
        
    #     # Get the two lists of colliding particles (partners)
    #     # particle_set_1 contains the first partner of each collision pair
    #     # particle_set_2 contains the second partner of each collision pair
    #     particle_set_1 = stopping_condition.particles(0)
    #     particle_set_2 = stopping_condition.particles(1)
        
        
    #     # Iterate over each collision event
    #     for i in range(len(particle_set_1)):
    #         p1 = particle_set_1[i]
    #         p2 = particle_set_2[i]
            
    #         # Retrieve current properties from the code (ph4) to ensure accuracy
    #         # Note: The particles in the set are 'copies' linked to the code, 
    #         # so accessing .position gets the live value.
            
    #         print(f"  Collision #{i+1} at t={gravity_code.model_time.in_(units.Myr)}")
    #         print(f"    Star 1 (ID: {p1.key}):")
    #         print(f"      Pos: {p1.position.in_(units.AU)}")
    #         print(f"      Vel: {p1.velocity.in_(units.kms)}")
    #         print(f"      Mass: {p1.mass.in_(units.MSun)}")
            
    #         print(f"    Star 2 (ID: {p2.key}):")
    #         print(f"      Pos: {p2.position.in_(units.AU)}")
    #         print(f"      Vel: {p2.velocity.in_(units.kms)}")
    #         print(f"      Mass: {p2.mass.in_(units.MSun)}")
            
    #         # Optional: Calculate relative velocity or distance
    #         rel_vel = (p1.velocity - p2.velocity).length()
    #         dist = (p1.position - p2.position).length()
    #         print(f"    Separation: {dist.in_(units.AU)}")
    #         print(f"    Rel Velocity: {rel_vel.in_(units.kms)}")
    #         print("--------------------------")
            
    #     break

        
        

Evolving to time: 1.0000213584301314 Myr
0.0993115101482 Myr
1
  Collision #100 at t=0.0993115101482 Myr
Evolving to time: 0.9000192225871468 Myr
0.100532741357 Myr
1
  Collision #100 at t=0.100532741357 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685273 Myr
1
  Collision #100 at t=0.100581685273 Myr
Evolving to time: 0.9000192225871468 Myr
0.1005816853 Myr
1
  Collision #100 at t=0.1005816853 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685326 Myr
1
  Collision #100 at t=0.100581685326 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685353 Myr
1
  Collision #100 at t=0.100581685353 Myr
Evolving to time: 0.9000192225871468 Myr
0.10058168538 Myr
1
  Collision #100 at t=0.10058168538 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685407 Myr
1
  Collision #100 at t=0.100581685407 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685434 Myr
1
  Collision #100 at t=0.100581685434 Myr
Evolving to time: 0.9000192225871468 Myr
0.100581685461 Myr
1
  Collision #10

[zuzia-linux:21606] OPAL ERROR: Server not available in file dpm/dpm.c at line 667
Exception ignored in: <function CodeInterface.__del__ at 0x7ac3532bb880>
Traceback (most recent call last):
  File "/home/zuzia/miniforge3/lib/python3.12/site-packages/amuse/rfi/core.py", line 861, in __del__
    self._stop()
  File "/home/zuzia/miniforge3/lib/python3.12/site-packages/amuse/rfi/core.py", line 968, in _stop
    self._stop_worker()
  File "/home/zuzia/miniforge3/lib/python3.12/site-packages/amuse/rfi/core.py", line 138, in __call__
    raise exceptions.CodeException(
amuse.support.exceptions.CodeException: Exception when calling function '_stop_worker', of code 'ph4Interface', exception was 'Error in code: no error message - code probably died, sorry.'


KeyboardInterrupt: 

In [ ]:

Mtot = cluster.mass.sum()
Rvir = cluster.virial_radius()
converter = nbody_system.nbody_to_si(Mtot, Rvir)

# --- integrator Huayno w SI ---
gravity_code = Huayno(converter)
gravity_code.particles.add_particles(cluster)
channel = gravity_code.particles.new_channel_to(cluster)
gravity_code.timestep = 0.01 | units.Myr




# --- potencjał Galaktyki w SI ---
MWG = MilkyWay_galaxy()  # domyślnie SI: m, kg, s

# --- detekcja kolizji ---
stopping_condition = gravity_code.stopping_conditions.collision_detection
stopping_condition.enable()

# # --- Bridge ---
# gravity = bridge.Bridge(use_threading=False)
# gravity.add_system(gravity_code, (MWG,))


# --- czas symulacji w SI ---
t_end = 100 * 1e6 * 365.25 * 24 * 3600 | units.s  # 10 Myr w sekundach
nsteps = 1000
dt = t_end / nsteps
time = 0.0 | units.s


t = []

# --- ewolucja ---
while time < t_end:
    gravity_code.evolve_model(time + dt)
    time = gravity_code.model_time
    print(time.in_(units.Myr))
    
    if stopping_condition.is_set():
        print("Collision detected at time:", time.in_(units.Myr))
        # tutaj można obsłużyć kolizję np. łączyć gwiazdy
    
   

gravity_code.stop()

In [ ]:
#evolving the system in the Milky Way potential
gravity_code = Huayno(converter)
gravity_code.particles.add_particles(cluster)
channel = gravity_code.particles.new_channel_to(cluster)
MWG = MilkyWay_galaxy()

gravity = bridge.Bridge(use_threading=False)
gravity.add_system(gravity_code, (MWG,) )

stopping_condition = gravity_code.stopping_conditions.collision_detection
stopping_condition.enable()



In [ ]:
t_end = 10 | nbody_system.time
nsteps = 10
dt = t_end/float(nsteps)
time = 0.0 | nbody_system.time
Nenc = 0
dEk_enc = zero    
dEp_enc = zero

t = []


In [ ]:
while time < t_end:
    gravity.evolve_model(time+dt)
    time = gravity.model_time
    if stopping_condition.is_set():
        print("Collision detected at time:", time.in_(units.Myr))
        
        